# Overview of Terraform Ops Notebook

## How to Use this Notebook
<p>
This notebook shows you how to easily explore Terraform features and functions by directly using the OCI Comand Line Interface (CLI).  First you will need to have the CLI tool installed on your machine and working.  Then you add a few elements to your config file as noted below, then Authenticate to your tenancy using the notebook cell provided.
</p>


## References


Resource Manager Stacks and Jobs [link](https://docs.oracle.com/en-us/iaas/Content/ResourceManager/Tasks/managingstacksandjobs.htm#stackstate  "Stacks and Jobs")


# Initialize Connection to Tenancy



In [ ]:
import oci
import pprint
# import subprocess
import json ,  pprint
from IPython.display import display, Math, HTML, Markdown
from pyUtils.runCLI import runCLI

display(HTML("<style>.container { width:100% !important; }</style>"))
# display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

try:
    tenancyProfile = oci.config.from_file()['target_profile']
except:
    print('A Target Profile not found in config file.  Using Default Profile')
    tenancyProfile = 'DEFAULT'

tenancyConfig = oci.config.from_file(profile_name=tenancyProfile)

config = ' --profile ' + tenancyConfig['auth_profile'] +  ' --auth security_token'
print('CLI Auth config string is: ', config)


# Terraform CLI


## Stacks

### Stack List 

In [ ]:

command = 'oci resource-manager stack list  --compartment-id ' \
        + tenancyConfig['target_compartment']  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stacks = json.loads(stdout)
except:
    print( 'Json convert error' ,stdout)

print('There were ' + str(len(stacks['data']) )+ '  Stacks Found')
stackDict = {}
for stack in stacks['data']:
    print(stack['display-name'])
    print('    Description: '  ,stack['description'])
    print('    Stack ID: ',stack['id'])
    print('    Time Created: ', stack['time-created'])
    print('    Status: ', stack['lifecycle-state'])
    print('    Terraform Version: ', stack['terraform-version'])

    stackDict[stack['display-name']] = stack['id']

    print('--')
# pprint.pprint(stackDict)



### Get Stack

In [ ]:
# targeStack = 'ocid1.ormstack.oc1.iad.aaaaaaaazuhcdkafhvuutnm3pojdzrsurbzagcgx2bv32elpbhekr2zbi2ma'
stackId = targetStack
command = ' oci resource-manager stack get --stack-id  ' + stackId +  config
print('Command: \n', command)
stdout, stderr = runCommand(command)
print(stderr)
stack = ''
try:
    stack = json.loads(stdout)
    print(stack['data']['display-name'])
    print('    Git Branch: ',stack['data']['config-source']['branch-name'])
    print('    Git Directory: ',stack['data']['config-source']['working-directory'])
    print('    Status: ',stack['data']['lifecycle-state'])
    print('    Time Created: ', stack['data']['time-created'])
    print('    Stack ID: ',stack['data']['id'])
except:
    print ('Error: ',stdout )
print('\n---------------')
pprint.pprint(stack)
# pprint.pprint(stackPlan)

### Get Stack's State

In [ ]:
file = "\"-\""
stackId = 'stack_id'
# stackId = targetStack
command = 'oci resource-manager stack get-stack-tf-state --stack-id  ' \
          + stackId + ' --file ' + file + config
print('Command: \n', command)
stdout, stderr = runCLI(command)

try:
    stackStates = json.loads(stdout)
except:
    print ('Error: ',stderr )
print('\nFetch State Complete, run cell below !')

In [ ]:
print('\nStack Resource Listing\n-------------\n')
stackOrder = ['httpmonitor', 'apigateway','apideployment', 'fnapp'  , 'fnfunc' 
              , 'id4d1hpkrsyu',   'eventrule'   , '.log.'   ]
# stackOrder = [ '.log.'   ]
resourceMap = {}
for instType in stackOrder:
    for stackState in stackStates['resources'] :  
#     try:
        for stackStateInstance in stackState['instances']:
            attributeId = stackStateInstance['attributes']['id']
            if instType in attributeId and  'httpmonitor' ==  instType  :
                print('HTTP Monitor')
                print('    ' ,  stackStateInstance['attributes']['display_name'])
                print('        ID:    ' ,  stackStateInstance['attributes']['id'])
                print('        Target:' ,  stackStateInstance['attributes']['targets'][0])
                print('        Path:  ' ,  stackStateInstance['attributes']['path'])
                myMon = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue
            elif instType in attributeId  and  instType ==  'id4d1hpkrsyu' :
                print('Object Storage Bucket')
                print('    ' , stackStateInstance['attributes']['name'])
                print('        ID:' , stackStateInstance['attributes']['id'])
                myOs = stackStateInstance
                resourceMap[stackStateInstance['attributes']['name']] = \
                            stackStateInstance['attributes']['id']
                continue                
            elif instType in attributeId  and  instType ==  'apideployment' :
                print('        API Gateway Deployment')
                print('            ' , stackStateInstance['attributes']['display_name'])
                print('                ID:' , stackStateInstance['attributes']['id'])       
                print('                Endpoint:' , stackStateInstance['attributes']['endpoint'])    
                print('                Function:' , stackStateInstance['attributes']['specification'][0]['routes'][0] ['backend'][0]['function_id'])
                deployPath = stackStateInstance['attributes']['path_prefix']  + \
                    stackStateInstance['attributes']['specification'][0]['routes'][0] ['path']   
                print('                Path:' , deployPath)
                myDeply = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue                  
            elif instType in attributeId  and  instType ==  'apigateway' :
                print('API Gateway')
                myGtw = stackStateInstance
                print('    ' , stackStateInstance['attributes']['display_name'])     
                print('        ID:' , stackStateInstance['attributes']['id'])                    
                print('        IP Address' , stackStateInstance['attributes']['ip_addresses'][0]['ip_address'])                    
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue                  
            elif instType in attributeId  and  instType ==  '.log.' :
                print('Log')
                print('    ' , stackStateInstance['attributes']['display_name'])
                print('         ID:' , stackStateInstance['attributes']['id'])
#                 print('         Type:' , stackStateInstance['attributes']['log_type'])
#                 print('         Service : ' , (myLog['attributes']['configuration'][0]['source'][0] ['service'] ))  
                myLog = stackStateInstance
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue                  
            elif instType in attributeId  and  instType ==  'eventrule' :
                print('Event Rule')
                print('    ' , stackStateInstance['attributes']['display_name'])   
                print('        ID:' , stackStateInstance['attributes']['id'])  
                myRule = stackStateInstance
                conditionsStr = stackStateInstance['attributes']   ['condition']
                conditions = json.loads(conditionsStr)
                print('        Bucket Name: ',  conditions['data']['additionalDetails']['bucketName'][0]  )
                print('        Filter  :    ', conditions ['data']['resourceName'][0]     )
                eventTypes = conditions['eventType'] 
                print('        Event Types')
                for eventType in eventTypes:
                    print('            ', eventType)
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue                  
            elif instType in attributeId  and  instType ==  'fnapp' :
                print('Application')
                print('    ' , stackStateInstance['attributes']['display_name'])   
                print('        ID:' , stackStateInstance['attributes']['id'])    
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
                continue  
            elif instType in attributeId  and  instType ==  'fnfunc' :
                print('        Function')
                print('            ' , stackStateInstance['attributes']['display_name'])    
                print('               ID:' , stackStateInstance['attributes']['id'])
                print( '               Image: '  ,stackStateInstance['attributes']['image'])
                configDict = stackStateInstance['attributes']['config']
                myFunct = stackStateInstance
                if len(configDict.keys()) > 0:
                    print('               Configuration:')
                    [print( '                    ' + key,':',value) for key, value in configDict.items()]
                resourceMap[stackStateInstance['attributes']['display_name']] = \
                            stackStateInstance['attributes']['id']
    print()

print('\n-------------')
with open('DevResourceMap.json', 'w') as convert_file:
     convert_file.write(json.dumps(resourceMap))
#     pprint.pprint (stackState['instances'])
print('DONE')

In [ ]:
with open('DevResourceMap.json') as f:
    data = f.read()
  
print("Data type before reconstruction : ", type(data))
      
# reconstructing the data as a dictionary
js = json.loads(data)
  
print("Data type after reconstruction : ", type(js))
print(js)

### Create Stack from file

<div class="alert alert-block alert-info">
<b>Tip:</b> To Check if Stack creation has completed, re-run the Stack List Report.</div>


In [ ]:
displayName = '\"Speedometer (CostApp.zip) Stack from file\"'
stackDescription = '\"Used CLI to create Stack \"'
configSource = '\"../terraform_files/CostApp.zip\"'
command = 'oci resource-manager  stack create --compartment-id  ' + targetComptId \
          + ' --config-source ' + configSource \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --terraform-version  "1.0.x" ' \
          +  config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Stack from Template

In [ ]:
variables =  '"{\\"Env\\": \\"Blue\\", \\"Log_group_id\\": \\"cecece\\"}"'
displayName = '\"Speedometer Stack 12/23 4pm \"'
sourceWorkingDirectory = '\"terraform_templates/speedometer/environments/Pre-prod_206255/\"'
stackDescription = '\"This Stack included .DS_Store files\"'
templateId = 'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaacgbvkgntyncqz2533escj7ongeomalp6jucsho3jaiia'
command = 'oci resource-manager stack create-from-template ' \
        + ' --compartment-id ' + targetComptId \
        + ' --description ' + stackDescription \
        + ' --display-name ' + displayName \
        + ' --template-id ' + templateId  \
        + ' --terraform-version  "1.0.x" ' \
        + config
#         + ' --variables ' + variables  \
#         + ' --working-directory ' + sourceWorkingDirectory \

print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Stack from Github

In [ ]:

sourceRepositoryUrl  =  '\"https://github.com/tr/truccr_oci-iac.git\"'
sourceBranchName = '\"split-fn-app-dev\"'
sourceWorkingDirectory = '\"terraform_templates/speedometer/environments/Pre-prod_206255\"'
displayName = '\"Dev - Test Speedometer Function Tag change test - Junk \"'
stackDescription = '\"This Stack is a Test \"'
configSourceProvider = 'ocid1.ormconfigsourceprovider.oc1.iad.aaaaaaaaqvci2yp3mgmkdggokooxgwwxhevnflp4r2yaie5hmmm7kqhibpoq'
command = 'oci resource-manager  stack create-from-git-provider --compartment-id  ' + targetComptId \
          + ' --config-source-configuration-source-provider-id ' + configSourceProvider  \
          + ' --config-source-repository-url ' + sourceRepositoryUrl \
          + ' --config-source-branch-name ' + sourceBranchName \
          + ' --description ' + stackDescription \
          + ' --display-name ' + displayName \
          + ' --config-source-working-directory ' + sourceWorkingDirectory \
          + ' --terraform-version  "1.0.x" ' \
          +  config
 
print('Command: \n', command)
# stdout, stderr = runCLI(command)
print(stderr)
try:
    stackOut = json.loads(stdout)
except:
    print ('Error: ',stdout )

targetStack = stackOut['data']['id']
print('Target Stack Name is:',stackOut['data']['display-name'] )
print('     ID:',targetStack)
print('---------------')
pprint.pprint(stackOut)
print('Done')

### Purge Stacks

In [ ]:

stackPurgeList = []
for name,id  in stackDict.items():
    if   'Junk' in name :
        print(name, ' - ' + id)
        stackPurgeList.append(id)
print('\nNumber of stacks in List: ',len(stackPurgeList))

In [ ]:
print('Beginning Purge')
for stack2Purge in stackPurgeList:
    command = 'oci resource-manager stack delete --force --stack-id ' + stack2Purge + config
    print('Command: \n', command)
    stdout, stderr = runCLI(command)
    print(stderr)
    print('Out: ', stdout)
print('Done')

## Jobs

### Stack Jobs

In [ ]:
stackId = 'stack_id'
# stackId = targetStack
command = 'oci resource-manager job list  --stack-id ' + stackId  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
stackJobs = ''
try:
    stackJobs = json.loads(stdout)
except:
    print( 'Json convert error' ,stdout)

for job in stackJobs['data']:
    print(job['display-name'])
    print('    Operation: ', job['operation'])
    print('    Status: ' , job['lifecycle-state'])
    print('    ID: ', job['id'])
    print('    Time Created: ', job['time-created'])
    print('    Time Finished: ', job['time-finished'])
    print('--')
print('----------\n')
pprint.pprint(stackJobs['data'])
print('Done')

### Stack Plan

In [ ]:
# stackId = 'stackDescription'
# stackId = targetStack
displayName = '\"TEST  ** Plan  changes with variables \"'
variables =  '"{\\"env\\": \\"Blue\\", \\"log_group_id\\": \\"cecece\\"}"'
command = ' oci resource-manager job create-plan-job --stack-id  ' + stackId \
        + ' --display-name ' + displayName  \
        +  config
#         + ' --variables ' + variables \
targetStack = ''
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
print('Creation of Plan Job Completed!!')
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
    print('    Time Created: ', stackPlan['data']['time-created'])
    print('    Job ID: ',stackPlan['data']['id'])
    print('    Stack ID: ',stackPlan['data']['stack-id'])
    targetStack = stackPlan['data']['stack-id']
except:
    print ('Error: ',stdout )
print('\n---------------')
print('Target Stack now is: ', targetStack )
# pprint.pprint(stackPlan)
print('Done')

### Stack Apply

In [ ]:
stackId = targetStack
displayName = '\"Test for bucket name change 2\"'
execPlanStrategy = 'AUTO_APPROVED'
# stackId = 'ocid1.ormstack.oc1.iad.aaaaaaaa3bnekl6352t6cfwnzkener6zddvdbsqis5fupd6ucm4aghxnh2uq'
command = ' oci resource-manager job create-apply-job --stack-id  ' + stackId \
          + ' --display-name ' + displayName  \
          + ' --execution-plan-strategy ' + execPlanStrategy  \
          +  config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
try:
    stackPlan = json.loads(stdout)
    print(stackPlan['data']['display-name'])
    print('    Git Branch: ',stackPlan['data']['config-source']['branch-name'])
    print('    Git Directory: ',stackPlan['data']['working-directory'])
    print('    Terraform Operation: ',stackPlan['data']['operation'])
    print('    Status: ',stackPlan['data']['lifecycle-state'])
except:
    print ('Error: ',stdout )
print('Done')



### Stack Destroy

<div class="alert alert-block alert-danger">
<b>Danger:</b> Running Stack Destroy will permanently remove OCI components.  Use caution when using.
</div>

In [ ]:
execPlanStrategy = 'AUTO_APPROVED'
stackId = ''
stackId = targetStack
command = ' oci resource-manager job create-destroy-job --stack-id  ' \
           + stackId  +  ' --execution-plan-strategy '  + execPlanStrategy  +     config
print('Do you want to destroy?(yes/no): ', stackId)
a = input()
if a == 'yes':
    print('yessire')
    print('Command: \n', command)
    stdout, stderr = runCLI(command)
    print(stderr)
    print(stdout)
print('Destroy Stack Cancelled !')

### Purge Jobs

**TBD**

## Templates

### PrivateTemplate List

In [ ]:
sortBy = ' --sort-by TIMECREATED --sort-order DESC  '
command = 'oci resource-manager template list --template-category-id 3 -c ' \
          + targetComptId + sortBy + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
templates = json.loads(stdout)

print('There were ' + str(len(templates['data']['items']) )+ '  Templates Found\n -------')
for template in templates['data']['items']:
    print(template['display-name'])
    print('    Description: ', template['description'] )
    print('    ID: ', template['id']  + '\n ---')
# pprint.pprint(templates['data'])

### Get Template

In [ ]:
templateId = \
'template_id'
command = 'oci resource-manager template get --template-id  ' + templateId  + config
print('Command: \n', command)
stdout, stderr = runCLI(command)
print(stderr)
template = json.loads(stdout)

pprint.pprint(template['data'])

### Get Template Config File

In [ ]:

file = "\"templatex.zip\""
templateId = 'template_id'
command = ' oci resource-manager template get-template-tf-config --template-id '+ templateId  + ' --file ' + file + config
print('Command: \n', command)
stdout,stderr = runCLI(command)
print(stderr)
try:
    template = json.loads(stdout)
except:
    print ('Error: ',stdout )

print('Done')

### Create Private Template

In [ ]:
# config1 = from_file(file_location="")
import base64
with open( r"CostApp.zip",'rb' ) as file:
    zipContents = file.read()
encodedZip = base64.b64encode(zipContents).decode('ascii')

zipfileupload = oci.resource_manager.models.CreateTemplateZipUploadConfigSourceDetails()
zipfileupload.config_source_type = 'ZIP_UPLOAD'
# zipfileupload.working_directory = 'MyTerraformScripts'
zipfileupload.zip_file_base64_encoded = encodedZip

resourcemanager = oci.resource_manager.models.CreateTemplateDetails()
resourcemanager.compartment_id = tenancyConfig['target_compartment']
resourcemanager.display_name = "Sample Template2"
resourcemanager.description = "Sample Temaplate2"
resourcemanager.template_config_source = zipfileupload

# def getCommandOutput(pCommand):
# Stack = oci.resource_manager.ResourceManagerClient(config1)
Template = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

Template.create_template(create_template_details=resourcemanager)


### Update Private Template

In [ ]:
import base64
with open( r"CostApp.zip",'rb' ) as file:
    zipContents = file.read()
encodedZip = base64.b64encode(zipContents).decode('ascii')
zipfileupload = oci.resource_manager.models.UpdateTemplateZipUploadConfigSourceDetails()
zipfileupload.config_source_type = 'ZIP_UPLOAD'
# zipfileupload.working_directory = 'MyTerraformScripts'
zipfileupload.zip_file_base64_encoded = encodedZip
resourcemanager = oci.resource_manager.models.UpdateTemplateDetails()
resourcemanager.compartment_id = tenancyConfig['target_compartment']
resourcemanager.display_name = "Sample Template2"
resourcemanager.description = "Sample Temaplate2x"
resourcemanager.
resourcemanager.template_config_source = zipfileupload
Template = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

testxx = Template.update_template(update_template_details=resourcemanager)
print(testxx.data)

### Create Stack from Template

In [ ]:

stackTemplateDetails=  oci.resource_manager.models.CreateStackTemplateConfigSourceDetails()
stackTemplateDetails.template_id=  \
   'ocid1.ormtemplate.oc1.iad.amaaaaaafmyzdhaacgbvkgntyncqz2533escj7ongeomalp6jucsho3jaiia'
stackTemplateDetails.config_source_type="TEMPLATE_CONFIG_SOURCE"
stackTemplateDetails.working_directory='terraform_templates'
# stackTemplateDetails.config_source_type='UNKOWN'

createStackDetails=oci.resource_manager.models.CreateStackDetails()
createStackDetails.config_source=stackTemplateDetails
createStackDetails.compartment_id = tenancyConfig['target_compartment']
createStackDetails.display_name = "Stack from Sample Template"
createStackDetails.description = "Stack from Sample Temaplate"
createStackDetails.terraform_version =  "1.0.x"
createStackDetails.variables={
            'env': 'xme',
            'log_group_id': 'wXfZ36ni34dituFB6MBo' }

stackTemplate = oci.resource_manager.ResourceManagerClient({'region': tenancyConfig['region']},
                                          signer=get_signer(tenancyConfig['auth_profile'] ))

create_stack_response=stackTemplate.create_stack(create_stack_details=createStackDetails)
# Get the data from response
print(create_stack_response.data)

